In [1]:
# Instal·lació de PySAT i les seves dependències
!pip install python-sat[aiger,approxmc,pblib]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.6 MB/s eta 0:00:00
  Created wheel for pypblib: filename=pypblib-0.0.4-cp310-cp310-linux_x86_64.whl size=2973438 sha256=0e7cf0f2a8d77de55fa564d2ef2114377314071b1c311720e6c5b911a0ce214e
  Stored in directory: /root/.cache/pip/wheels/a8/3d/9c/b980a09d6084eaeb9b336edbe20371c53c2550226daeee2025
Successfully built pypblib


In [2]:
from pysat.formula import CNF
from pysat.solvers import Glucose3
from pysat.card import CardEnc, EncType
from IPython.display import HTML
import numpy as np
import time
import math

In [3]:
nomsat_a_nomreal = {}
nomreal_a_nomsat = {}

nomsat = 1

for fila in range(1, 9):
  for columna in range(1,9):
      nomreal = int(str(fila)+str(columna))
      nomsat_a_nomreal[nomsat] = nomreal
      nomreal_a_nomsat[nomreal] = nomsat
      nomsat += 1

In [4]:
def obtener_fila(fila):
  indexos = []
  for columna in range(1,9):
    nomreal = int(str(fila)+str(columna))
    indexos.append(nomreal_a_nomsat[nomreal])
  return indexos

In [5]:
def obtener_columna(columna):
  indexos = []
  for fila in range(1, 9):
    nomreal = int(str(fila)+str(columna))
    indexos.append(nomreal_a_nomsat[nomreal])
  return indexos

In [39]:
def obtener_diagonales():
 indexos=[]

 # Diagonales izquierda-derecha
 for n in range (0,7):
  indexos1=[]
  for i in range (n,8):
   j=i-n
   nomreal = int(str(i+1)+str(j+1))
   indexos1.append(nomreal_a_nomsat[nomreal])
   #print(nomreal)
  indexos.append(indexos1)
  #print("------")

 for n in range (1,7):
  indexos2=[]
  for i in range (n,8):
    j=i-n
    nomreal = int(str(j+1)+str(i+1))
    indexos2.append(nomreal_a_nomsat[nomreal])
    #print (nomreal)
  indexos.append(indexos2)
  #print("------")

 #Diagonales de derecha-izquierda
 for n in range(1,8):
  indexos3=[]
  for i in range(0,8-(7-n)):
   j=abs(i-n)
   nomreal = int(str(i+1)+str(j+1))
   indexos3.append(nomreal_a_nomsat[nomreal])
   #print(nomreal)
  indexos.append(indexos3)
  #print("----")

 for n in range(6,0,-1):
  indexos4=[]
  for i in range(0,8-(7-n)):
   j=abs(i-n)
   nomreal = int(str(abs(j-7)+1)+str(abs(i-7)+1))
   indexos4.append(nomreal_a_nomsat[nomreal])
   #print(nomreal)
  indexos.append(indexos4)
  #print("----")

 return indexos

In [52]:
def obtener_filas_y_columnas():
  sat_restriccion = []

  # Restriccion tipo fila
  for fila in range(1, 9):
     sat_restriccion.append(obtener_fila(fila))
  # Restriccion tipo columna
  for columna in range(1, 9):
    sat_restriccion.append(obtener_columna(columna))

  return sat_restriccion

In [53]:
def obtener_formula_general():

  cnf = CNF()

  #Obtiene las restricciones de fila y columna
  for sat in obtener_filas_y_columnas():
    cnf.extend(CardEnc.equals(lits=sat, bound=1, encoding=EncType.pairwise))

  #Obtiene las restricciones de las diagonales
  for sat_dig in obtener_diagonales():
    cnf.extend(CardEnc.atmost(lits=sat_dig, bound=1, encoding=EncType.pairwise))

  return cnf

In [54]:
# Creamos el tablero
tablero = np.zeros((8, 8), dtype=int)
formula_cnf = obtener_formula_general()

# Solucionamos
solver = Glucose3(bootstrap_with=formula_cnf.clauses)
temps_inici = time.time()
sat = solver.solve()
temps_fi = time.time()

# Imprimimos el tablero si hay solución
if (sat):
  model = [e for e in solver.get_model() if e > 0] # Sólo interesan los positivos
  for pos in model:
    real = nomsat_a_nomreal[pos]
    tablero[int(str(real)[0]) - 1, int(str(real)[1]) - 1] = str(1)
  print("\nTablero solucionado:")
  print(tablero)
  print("\nProblema solucionado en %f milisegundos" % ((temps_fi - temps_inici)*1000))
else:
  print("El problema no tiene solución!!")



Tablero solucionado:
[[0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0]]

Problema solucionado en 0.307560 milisegundos
